# Datalogging

## code.py: 

```python
import time
import board
import digitalio
import neopixel

from adafruit_circuitplayground.express import cpx

try:
    #check the switch state again
    readonly_fs = cpx.switch
    if readonly_fs:
        #CODING MODE
        #setup the NeoPixels for light display
        pixels = neopixel.NeoPixel(board.NEOPIXEL, 10, brightness=.2)
        pixels.fill((0,0,0))
        pixels.show()
        num_pixels = len(pixels)
        while True:
            pixels.fill((0,0,0))
            for i in range(num_pixels):
                pixels[i] = (0,0,255) #bright BLUE
                time.sleep(0.5)
    else:    
        with open("/temperature.txt", "a+") as fp:
            while True:
                temp = cpx.temperature
                # do the C-to-F conversion here if you would like
                fp.write('{0:f}\r\n'.format(temp))
                fp.flush()
                cpx.red_led = not cpx.red_led
                print("logging data")
                time.sleep(1.0)
except Exception as e:
    print("Caught Error: %r" % e)
    delay = 0.5
    if e.args[0] == 28:
        delay = 0.25
    while True:
        cpx.red_led = not cpx.red_led
        time.sleep(delay)
```

## boot.py

```python
import board
import digitalio
import storage

switch = digitalio.DigitalInOut(board.D7)
switch.direction = digitalio.Direction.INPUT
switch.pull = digitalio.Pull.UP

# If the D7 switch is DOWN (when USB on left, 
# then it is PULLED HIGH, switch.value == True) 
# and we in CODING MODE.
# If it is UP, then we are in DATALOGGING MODE!
readonly = switch.value
if readonly:
    print("Entering CODING_MODE")
    storage.remount("/", True)
else:
    print("Entering DATALOGGING_MODE")
    storage.remount("/", False)   
```